In [12]:
# se instalan las dependencias necesarias
! pip install -r requirements.txt


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
#Se importan las librerias necesarias 
import pandas as pd

In [14]:
# Se importa archivo con el estock actual del negocio

file_stockActual = '08_11/Inventario_0811.xlsx'
inventario_actual = pd.read_excel(file_stockActual)
#Se establecen columas a utilizar
inventario_actual = inventario_actual[['Código', 'Producto', 'Existencia', 'P. Costo', 'P. Venta', 'Departamento']]

#Se renombran las columnas para facilitar el manejo
inventario_actual.columns = ['code', 'product', 'qty', 'cost', 'price', 'department']

#Se eliminan caracteres especiales no relevantes
inventario_actual['code'] = inventario_actual['code'].str.replace(' ', '').str.replace('.', '').astype(str)
inventario_actual['qty'] = inventario_actual['qty'].str.replace('$', '').str.replace('.', '').astype(str)
inventario_actual['cost'] = inventario_actual['cost'].str.replace('$', '').str.replace('.', '').astype(str)
inventario_actual['price'] = inventario_actual['price'].str.replace('$', '').str.replace('.', '').astype(str)

#se definen los tipos de datos de las columnas
inventario_actual['code'] = inventario_actual['code'].astype('string')
inventario_actual['product'] = inventario_actual['product'].astype('string')
inventario_actual['qty'] = pd.to_numeric(inventario_actual['qty'], errors='coerce')
inventario_actual['cost'] = pd.to_numeric(inventario_actual['cost'], errors='coerce')
inventario_actual['price'] = pd.to_numeric(inventario_actual['price'], errors='coerce')


print(inventario_actual)


            code                              product   qty   cost  price  \
0      175002710  Pisco Mistral Nobel Barrica Tostada   7.0   9622  13000   
1        1801828                           Budwuaisen  17.0    890   1000   
2     2490238841                       Red Bull 473ml   9.0   1650   2200   
3       30052765                    Ocb 1 1/4 Premium  21.0    439   1000   
4       30068926                            Ocb Xpert  14.0    412   1000   
..           ...                                  ...   ...    ...    ...   
890     90446849          >Red Bull Red Edition 250ml   3.0   1240   1800   
891     90453823            Red Bull Sugar Free 473ml  10.0   1997   2600   
892     90454707             Redbull Sabor Fruta250ml  20.0   1126   1800   
893     90456695                Red Bull Winter 250ml  24.0   1240   1800   
894  96749021345                Whiskey Evan Williams   3.0  13000  18000   

          department  
0            Licores  
1           Cervezas  
2    J

In [15]:
#Archivos de inventario Capturados con el lector de codigos de barras android

file1 = '08_11/Manuel_01.xls'
file2 = '08_11/Manuel_02.xls'
file3 = '08_11/Roberto.xls'
file4 = '08_11/Elena.xls'

#Se leen los archivos de inventario 
inventario1 = pd.read_excel(file1)
inventario2 = pd.read_excel(file2)
inventario3 = pd.read_excel(file3)
inventario4 = pd.read_excel(file4)

#Se concatenan los archivos de inventario en uno solo
TotalInventario = pd.concat([inventario1, inventario2, inventario3, inventario4])

#Se establecen las columnas a utilizar
TotalInventario = TotalInventario[['Código de Barras:', 'Artículo:', 'Unidades:']]

#se cambian los nombres de las columnas para facilitar el manejo de los datos
TotalInventario.columns = ['code', 'product', 'qty']

#se convirtene los datos de la columna 'code' a string y 'qty' a enteros
TotalInventario['code'] = TotalInventario['code'].astype(str)
TotalInventario['qty'] = pd.to_numeric(TotalInventario['qty'], errors='coerce', downcast='integer')

print(TotalInventario)


              code                         product  qty
0    7501233712713  Tequila blanco el merendero 1L    2
1    5000289020701             Gordons dry gin 750    2
2    5000291026548           Tanque Ray dark berry    1
3    5000281015248                Tanqueray london    3
4    7640175740047                 Bomba vapor gin    2
..             ...                             ...  ...
180  7801620006631          cachantun más granada     4
181  7804646000256                   mister Big 2L    1
182  7804603921082                    Sandy Mac 1L    2
183  5010408252074                   Sandy Mac 750    2
184  6932757154818      pañuelo desechable care up   40

[572 rows x 3 columns]


In [16]:
# se comprueba si los codigos de totalinventario estan en inventario_actual y se almacenan los codigos faltantes en una lista
Faltantes = []
exclude = ['8501110080255','8804374087016','7804300155889', '4841676004925', '11624822', '9807181688642', '3827372569853', '5571147', '2722764', '7707111', '2228620006600', ]

for i in TotalInventario['code']:
     if i not in exclude:
          if i not in inventario_actual['code'].values:
               Faltantes.append(i)

#se agrupan los codigos unicos de totalinventario y se suman los 'qty' de cada uno de ellos para obtener el total de cada producto en el inventario
TotalInventario = TotalInventario.groupby('code').sum().reset_index()

#se exporta totalinventario a un archivo excel
TotalInventario.to_excel('total_inventario.xlsx', index=False, engine='openpyxl') 


In [17]:
#se abre el archivo original de inventario para usarlo como plantilla
form_file = pd.read_excel(file_stockActual)

#se establece la columna 'existencia' a 0 para todos los codigos
form_file['Existencia'] = 0

# se remplaza la columna 'existencia' con los valores de 'qty' de totalinventario (realizado en el paso anterior)
for i in range(len(TotalInventario)):
     form_file.loc[form_file['Código'] == TotalInventario['code'][i], 'Existencia'] = TotalInventario['qty'][i]


#se eliminan "$" y "." de la columna 'P. Costo' y 'P. Venta' y P.Mayoreo y se convierten a numericos
form_file['P. Costo'] = form_file['P. Costo'].str.replace('$', '').str.replace('.', '').astype(int)
form_file['P. Venta'] = form_file['P. Venta'].str.replace('$', '').str.replace('.', '').astype(int)
form_file['P. Mayoreo'] = form_file['P. Mayoreo'].str.replace('$', '').str.replace('.', '').astype(int)

# se establece a numerico la columna Inv. Mínimo y Inv. Máximo
form_file['Inv. Mínimo'] = pd.to_numeric(form_file['Inv. Mínimo'], errors='coerce', downcast='integer')
form_file['Inv. Máximo'] = pd.to_numeric(form_file['Inv. Máximo'], errors='coerce', downcast='integer')

#se exporta el archivo con el inventario actualizado listo para ser cargado a la base de datos
form_file.to_excel('Inventario_0811_actualizado.xlsx', index=False, engine='openpyxl')

In [18]:
print(inventario_actual.dtypes)
print(inventario_actual.iloc[0])

code          string[python]
product       string[python]
qty                  float64
cost                   int64
price                  int64
department            object
dtype: object
code                                    175002710
product       Pisco Mistral Nobel Barrica Tostada
qty                                           7.0
cost                                         9622
price                                       13000
department                                Licores
Name: 0, dtype: object


In [22]:
#se calculan los productos que tienen diferencia de cantidades de inventario entre el inventario actual y el inventario capturado

stock_actual = inventario_actual
inventario_actualizado = TotalInventario

#se renombran las columas para facilitar el manejo
inventario_actualizado = inventario_actualizado[['code', 'product', 'qty']]
inventario_actualizado.columns = ['code', 'a_product' ,'a_qty']

inventario_actualizado['code'] = inventario_actualizado['code'].astype('string')
inventario_actualizado['a_product'] = inventario_actualizado['a_product'].astype('string')
inventario_actualizado['a_qty'] = pd.to_numeric(inventario_actualizado['a_qty'], errors='coerce')

#se unen los dos dataframes para comparar las cantidades de inventario
comparacion = pd.merge(stock_actual, inventario_actualizado, on='code', how='inner')

#se calcula la diferencia de cantidades de inventario
comparacion['diff'] = comparacion['qty'] - comparacion['a_qty']

#se eliminan las filas que no tienen diferencias y las que tienen cantidades negativas
comparacion = comparacion[comparacion['diff'] != 0]
comparacion = comparacion[comparacion['diff'] > 0]

#se crea una columna con la cantidad de dinero en diferencia de inventario
comparacion['diff_cost'] = comparacion['diff'] * comparacion['cost']

#se eliminan las columnas que no son necesarias para el reporte
comparacion = comparacion[['code', 'product','department' , 'qty', 'a_qty', 'diff', 'cost', 'diff_cost']]
comparacion.columns = ['Código', 'Producto','Departamento', 'Cantidad Actual', 'Cantidad inventario', 'Diferencia', 'Costo', 'Diferencia en Costo']

#se ordena el reporte por departamento
comparacion = comparacion.sort_values(by='Departamento')

#se exporta el dataframe a un archivo excel
comparacion.to_excel('reporte_faltantes.xlsx', index=False, engine='openpyxl')
